# Recipe retrival!

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd 

## Getting the recipe links!
Scraping the website for the top recipes by ingredient over all ingredients. 

In [164]:
headers = {'user-agent': 'Pindale UVA HooHacks Project (mwp8zy@virginia.edu)'}
Base_url = 'https://tasty.co/ingredient' #I think I can get all of the recipes using recipe by ingredient
r = requests.get(Base_url, headers=headers)
soup = BeautifulSoup(r.content, 'html.parser')


In [165]:
test = soup.find_all(type="application/ld+json")


In [167]:
holder = test[0]
x=str(holder)[35:16902] #This is the part that is a json
import json
y = json.loads(x)
links_to_recipes_by_ing = [i['url'] for i in y["itemListElement"]]

In [317]:
links_to_recipes_by_ing[0:2] #Just peaking to see if it worked!

['https://tasty.co/ingredient/all-purpose-flour',
 'https://tasty.co/ingredient/allspice']

In [190]:
recipe_links = []
for link in links_to_recipes_by_ing:
    Base_url = link
    r = requests.get(Base_url, headers=headers)
    soup = BeautifulSoup(r.content, 'html.parser')
    test = soup.find_all(type="application/ld+json")
    try:
        holder = test[0]  
        holder
        x=str(holder)[35:-9]
        y = json.loads(x)
        recipe_links += [i['url'] for i in y["itemListElement"]]
    except:
        pass

Here I check for repeats by looking at the length of tthe list and the set

In [191]:
len(recipe_links)

3794

In [194]:
len(list(set(recipe_links)))

2098

Getting rid of the repeats

In [195]:
final_recipe_links = list(set(recipe_links))

## Scraping the instructions!

In [246]:
instructions = []
for link in final_recipe_links:
    Base_url = link
    r = requests.get(Base_url, headers=headers)
    soup = BeautifulSoup(r.content, 'html.parser')
    test = soup.find_all(class_="xs-mb2")
    instructions.append(test[2].get_text())

In [249]:
len(instructions)

2098

## Scraping the ingrdients!

In [315]:
def grab_ingredients(tags):
    """Input the list of tags from scraping
    Output the list of ingredients"""
    ls = []
    for tag in tags:
        ls.append(tag.get_text())
    ing = []
    for i in range(len(ls)):
        if ls[i].startswith('featured'):
            pass
        elif ls[i].startswith('Calories'):
            break
        else:
            ing.append(ls[i])
    return(ing)




ingredients = []
for link in final_recipe_links:
    Base_url = link
    r = requests.get(Base_url, headers=headers)
    soup = BeautifulSoup(r.content, 'html.parser')
    test = soup.find_all(class_="xs-mb1")
    ingredients.append(grab_ingredients(test))

In [316]:
len(ingredients)

2098

## Creating a dataframe and saving to a csv for later

In [318]:
data = pd.DataFrame({'recipe_link':final_recipe_links, 'ingredients':ingredients, 'instructions':instructions})

In [320]:
data.to_csv('scraped_recipe_data')